In [1]:
import calibrate as cal
import collect as col
import post_process as pp

ModuleNotFoundError: No module named 'calibrate'

In [ ]:
p_tot = col.run_total_power_int( 1024, 49.6, 2.048e6, 91.3e6, 5)
print('Uncalibrated total power measurement: {}'.format(p_tot))

In [ ]:
f, p = col.run_spectrum_int(1024, 49.6, 2.048e6, 91.3e6, 5)
fig = pp.plot_spectrum(f, p)

In [ ]:
f, p = col.run_fswitch_int(2048, 49.6, 2.048e6, 89.09e6, 89.11e6, 2.0, fswitch=1)
fig = pp.plot_spectrum(f, p)